In [1]:
#!pip install datasets
#!pip install transformers
#!pip install evaluate
#!pip install accelerate - U
#!pip install transformers[torch]
#!pip install peft
#!pip install torch torchvision transformers datasets

In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

from datasets import load_dataset
from datasets import Dataset
from transformers import TrainingArguments, Trainer, GenerationConfig
from transformers import BertModel, BertTokenizer, AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import evaluate
import pandas as pd
import numpy as np
#anwenden des modells auf das erste fullText element

import datetime
import torch
import methods as mp
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [2]:
model_name = "distilbert-base-uncased"
path_social_science = '/Users/merlin/Desktop/BA/Bachelorabeit Uni Leipzig/programming/dhox/data/1880_nopsy.jsonl'
dataset =  pd.read_json(path_social_science, lines=True)

In [3]:
#loade distilBert a smaller version of bert für kürzere Laufzeit
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [4]:
#!pip install torch torchvision transformers datasets

In [6]:
dataset.columns

Index(['creator', 'datePublished', 'docSubType', 'docType', 'doi', 'fullText',
       'id', 'identifier', 'isPartOf', 'issueNumber', 'keyphrase', 'language',
       'outputFormat', 'pageCount', 'pageEnd', 'pageStart', 'pagination',
       'provider', 'publicationYear', 'publisher', 'sequence', 'tdmCategory',
       'title', 'url', 'volumeNumber', 'wordCount', 'unigramCount',
       'bigramCount', 'trigramCount', 'sourceCategory', 'abstract'],
      dtype='object')

In [7]:
train_data = pd.DataFrame()
for i in range(2010,2021):
    data = dataset[dataset['datePublished'].str[:4] == str(i)]
    train_data= pd.concat([train_data, data], ignore_index=True)
train_data['datePublished'] = pd.to_datetime(train_data['datePublished'], errors='coerce')


In [8]:
train_data = train_data[train_data['datePublished'].dt.year == 2020]
len(train_data)

480

In [9]:
train_data = train_data[['fullText', 'id', 'datePublished','sourceCategory']]

In [10]:
train_data["fullText"] = train_data["fullText"].apply(mp.join_strings)
train_data['fullText']

3350    INTRODUCTION TO THE SYMPOSIUM ON COVID-19, GLO...
3351    55\nCapitalist and Communal Foundations in The...
3352    RESEARCH ARTICLE\nCopyright Donets EV, Chudino...
3353    Creative Commons Non Commercial CC BY-NC: This...
3354    Quality of Life for Women with Human\nPapillom...
                              ...                        
3825    حول الشباب والهوية الوطنية في الشرط الفلسطيني ...
3826    Research-based language learning through \nfra...
3827    Journal of Ethnic and Cultural Studies Copyrig...
3828    42 RCC Perspectives Women and Energy 43 Introd...
3829    Book Reviews 321 Johnhenry Gonzalez, Maroon Na...
Name: fullText, Length: 480, dtype: object

In [11]:

train_data["token_count"] = train_data["fullText"].apply(lambda x: len(tokenizer(x)["input_ids"]))



Token indices sequence length is longer than the specified maximum sequence length for this model (4359 > 512). Running this sequence through the model will result in indexing errors


In [13]:
train_data['token_count'].sum()


4610471

In [15]:
train_data["fullText"] = train_data["fullText"].apply(mp.join_strings)
train_data["token_count"] = train_data["fullText"].apply(lambda x: len(tokenizer(x)["input_ids"]))
train_data['token_count'].sum()

4610471

In [16]:
len(dataset)

16912

In [ ]:
encoded_dataset = train_data.map(preprocess_function, batched=True)

In [50]:
chunked_df = preprocess_with_labels(train_data)

Processing chunks: 100%|██████████| 1/1 [00:03<00:00,  3.58s/it]


In [51]:
chunked_dataset = chunked_df
print(chunked_dataset.column_names)

['input_ids', 'attention_mask', 'labels']


In [52]:
chunked_dataset = chunked_dataset.remove_columns(
    [col for col in chunked_dataset.column_names if col not in ["input_ids", "attention_mask", "labels"]]
)
print(chunked_dataset.column_names)

['input_ids', 'attention_mask', 'labels']


In [53]:
chunked_dataset.column_names

['input_ids', 'attention_mask', 'labels']

In [44]:
grouped_dataset = group_chunks(chunked_df)


NameError: name 'group_chunks' is not defined

In [33]:
len(grouped_dataset)

3350

In [43]:
len(chunked_dataset)


480

In [33]:
from torch.utils.data import default_collate
from transformers import DataCollatorForLanguageModeling

class GroupedDataCollator(DataCollatorForLanguageModeling):
    def torch_call(self, features):
        # Flatten chunks within each document
        flat_features = []
        for f in features:
            for ids, mask in zip(f["input_ids"], f["attention_mask"]):
                flat_features.append({"input_ids": ids, "attention_mask": mask})
        return super().torch_call(flat_features)


In [44]:
from transformers import DataCollatorForLanguageModeling

# Create a data collator that dynamically masks tokens
data_collator = DataCollatorForLanguageModeling(
    tokenizer=DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased"),
    mlm=True,
    mlm_probability=0.15,   # 15% masking, standard BERT objective
)

In [61]:
# Assuming `dataset` is your Hugging Face Dataset from preprocess_funktion_fast_list_of_lists
chunked_dataset = chunked_dataset.remove_columns([c for c in chunked_dataset.column_names if c not in ["input_ids","attention_mask","labels"]])
split_dataset = chunked_dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]


train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

{'input_ids': tensor([  101, 17871,  1997,  2740,  1004,  4200,  5285,  1012,  1017,  1010,
          2053,  1012,  1017,  1010,  4903,  1012, 24398,  1516, 25797,  9193,
          1024,  2184,  1012,  5401, 22932,  1013,  1054, 14227,  1012, 12609,
          1012,  7886,  2629,  1075, 12609,  2118,  1997,  3516,  2811, 20300,
         18046,  1998,  4975, 20129,  3802, 15006,  1999,  3433,  2000,  2966,
          3691,  1024,  2019,  4357,  2817,  1997,  2308,  2091,  8715, 13010,
          6864,  1054,  1012,  7305,  1998, 11496, 10635,  1999,  2023,  3720,
          1010,  2057,  7475,  2008,  2091,  8715,  1006, 16233,  1007, 13010,
          6224,  2000, 18793,  1999,  2966,  4654, 29206,  9623,  2024,  9996,
         10959,  2011,  2037,  9501,  1997,  5022,  1998, 11572,  1012,  2000,
          4337,  2023,  3291,  1010,  2057,  2424,  2008,  2070, 13010,  1010,
          4919, 10756,  1997,  2336,  2007, 16233,  1010, 28667,  6279, 22139,
          2037,  3802, 15006,  2083,  2

In [55]:
from transformers import DistilBertForMaskedLM

model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

In [71]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./dapt-distilbert",
    overwrite_output_dir=True,
    logging_dir="./logs",
    evaluation_strategy="steps",
    eval_steps=5,
    save_strategy="steps",
    save_steps=5,
    logging_steps=5,
    learning_rate=5e-5,
    weight_decay=0.01,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    warmup_steps=5,
    save_total_limit=2,
    fp16=False,             # 👈 CPU safe
    report_to="none"
)


training_args.use_cpu=True

/Users/merlin/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [72]:
from transformers import Trainer
import wandb

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased"),
    data_collator=data_collator
)

/var/folders/jb/jz3xgqts0zx_lxl_9ylrxrv80000gn/T/ipykernel_53071/1409845953.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [73]:
trainer.train()

Step,Training Loss,Validation Loss
5,1.369600,2.544536
10,2.414500,2.689576
15,2.630500,2.777219
20,2.691400,2.520288
25,2.510400,2.560085
30,2.508800,2.672796
35,2.492700,2.535910
40,2.485200,2.559680
45,2.462100,2.613721
50,2.501200,2.441816


/Users/merlin/miniconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/merlin/miniconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/merlin/miniconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/merlin/miniconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/merlin/miniconda3/lib/python3.12/site-package

TrainOutput(global_step=57, training_loss=2.434505169851738, metrics={'train_runtime': 139.2839, 'train_samples_per_second': 3.274, 'train_steps_per_second': 0.409, 'total_flos': 60447887179776.0, 'train_loss': 2.434505169851738, 'epoch': 1.0})

In [70]:

# Evaluate
eval_results = trainer.evaluate()
import numpy as np
print("Eval loss:", eval_results["eval_loss"])
print("Perplexity:", np.exp(eval_results["eval_loss"]))


/Users/merlin/miniconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Eval loss: 2.4412734508514404
Perplexity: 11.487660403031047


In [32]:
#save trained model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [33]:
# Load them back when needed
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
model = DistilBertForSequenceClassification.from_pretrained("./fine_tuned_model")
tokenizer = DistilBertTokenizer.from_pretrained("./fine_tuned_model")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at ./fine_tuned_model and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
tensorboard --logdir=./logs

SyntaxError: cannot assign to expression (2911169554.py, line 1)

In [108]:
from transformers import DistilBertTokenizer

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Preprocessing function
def preprocess_function(examples):
    return tokenizer(examples['sentence'], padding="max_length", truncation=True)

# Apply preprocessing
encoded_dataset = raw_dataset.map(preprocess_function, batched=True)

# Check processed data
print(encoded_dataset['train'][0])

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0, 'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0